# High-ROC AUC Tabular Neural Net (PyTorch) — Stratified K-Fold

This notebook is auto-generated to target **ROC AUC** for the Kaggle Playground S5E8 "Binary Classification with a Bank Dataset".  
It uses:
- A **neural network** (PyTorch) with **categorical embeddings** + **numeric standardization**
- **Stratified K-Fold** CV with **OOF** predictions
- **Early stopping** + **LR scheduler**
- Organized, timestamped **run folders** with metrics, models, logs, and **graphs**

> Tip: Put `train.csv`, `test.csv` in the working directory. Modify the **CONFIG** section if your columns differ.


In [9]:
# =========================
# CONFIG
# =========================
COMPETITION_NAME = "playground-series-s5e8"  # label only
ID_COL = "id"
TARGET_COL = "y"

# Files
TRAIN_PATH = "playground-series-s5e8/train.csv"
TEST_PATH = "playground-series-s5e8/test.csv" 

# CV
N_SPLITS = 5
RANDOM_SEED = 2025

# Model / Training
BATCH_SIZE = 4096
EPOCHS = 100
PATIENCE = 12                # early stopping
BASE_LR = 1e-3
WEIGHT_DECAY = 1e-5
HIDDEN_LAYERS = [512, 256, 128]
DROPOUT = 0.2
USE_CLASS_WEIGHTS = True

# Output (auto-filled below)
RUN_STAMP = None
RUN_DIR = None               # root for this run
FIGS_DIR = None
FOLDS_DIR = None
LOGS_DIR = None
MODELS_DIR = None
ARTIFACTS_DIR = None

VERBOSE_EVERY = 1
SAVE_OOF = True


In [10]:
# =========================
# IMPORTS & FOLDERS
# =========================
import os, gc, math, random, json, time
from pathlib import Path
from typing import List, Tuple, Dict

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, RocCurveDisplay
from sklearn.preprocessing import StandardScaler, LabelEncoder

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(RANDOM_SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Prepare organized run folders
RUN_STAMP = time.strftime("%Y-%m-%d_%H-%M-%S")
RUN_DIR = Path(f"runs/{RUN_STAMP}")
FIGS_DIR = RUN_DIR / "figs"
FOLDS_DIR = RUN_DIR / "folds"
LOGS_DIR = RUN_DIR / "logs"
MODELS_DIR = RUN_DIR / "models"
ARTIFACTS_DIR = RUN_DIR / "artifacts"

for d in [RUN_DIR, FIGS_DIR, FOLDS_DIR, LOGS_DIR, MODELS_DIR, ARTIFACTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# Save config snapshot
cfg = dict(
    COMPETITION_NAME=COMPETITION_NAME,
    ID_COL=ID_COL,
    TARGET_COL=TARGET_COL,
    TRAIN_PATH=TRAIN_PATH,
    TEST_PATH=TEST_PATH,
    N_SPLITS=N_SPLITS,
    RANDOM_SEED=RANDOM_SEED,
    BATCH_SIZE=BATCH_SIZE,
    EPOCHS=EPOCHS,
    PATIENCE=PATIENCE,
    BASE_LR=BASE_LR,
    WEIGHT_DECAY=WEIGHT_DECAY,
    HIDDEN_LAYERS=HIDDEN_LAYERS,
    DROPOUT=DROPOUT,
    USE_CLASS_WEIGHTS=USE_CLASS_WEIGHTS,
    RUN_STAMP=RUN_STAMP,
)
with open(RUN_DIR / "config.json", "w") as f:
    json.dump(cfg, f, indent=2)

print("Run folder:", RUN_DIR.as_posix())


Device: cpu
Run folder: runs/2025-08-12_09-12-57


In [11]:
# =========================
# LOAD DATA
# =========================
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)
print("Train:", train.shape, "Test:", test.shape)

assert TARGET_COL in train.columns, f"TARGET_COL '{TARGET_COL}' missing"
assert ID_COL in train.columns, f"ID_COL '{ID_COL}' missing"
assert ID_COL in test.columns, f"ID_COL '{ID_COL}' missing in test"

feature_cols = [c for c in train.columns if c not in [TARGET_COL, ID_COL]]
missing_in_test = [c for c in feature_cols if c not in test.columns]
assert not missing_in_test, f"Missing features in test: {missing_in_test}"

y = train[TARGET_COL].values
print("Target positive rate:", y.mean().round(6))


Train: (750000, 18) Test: (250000, 17)
Target positive rate: 0.120651


### Feature typing & preprocessing

- **Categoricals**: object dtype or low-cardinality integers → label-encoded + **embeddings**  
- **Numerics**: standardized via `StandardScaler`  
- Rare categories combined into `__RARE__` for stability


In [12]:
# =========================
# FEATURE TYPING
# =========================
obj_cols = [c for c in feature_cols if train[c].dtype == 'object']
lowcard_int_cols = [c for c in feature_cols 
                    if str(train[c].dtype).startswith('int') and train[c].nunique() <= 30]
cat_cols = sorted(list(set(obj_cols + lowcard_int_cols)))
num_cols = sorted([c for c in feature_cols if c not in cat_cols])
print(f"Categoricals ({len(cat_cols)}):", cat_cols[:20])
print(f"Numerics ({len(num_cols)}):", num_cols[:20])

# Rare category handling
RARE_NAME = "__RARE__"
MIN_CAT_COUNT = 25

def apply_rare(series: pd.Series, min_count: int = MIN_CAT_COUNT) -> pd.Series:
    counts = series.value_counts()
    rare = counts[counts < min_count].index
    return series.where(~series.isin(rare), RARE_NAME)

# Label encoding fit on full (train+test) to keep indices aligned
encoders = {}
for c in cat_cols:
    s_tr = apply_rare(train[c].astype(str))
    s_te = apply_rare(test[c].astype(str))
    le = LabelEncoder()
    le.fit(pd.concat([s_tr, s_te], axis=0).fillna("NA"))
    encoders[c] = le
    train[c] = le.transform(s_tr.fillna("NA"))
    test[c]  = le.transform(s_te.fillna("NA"))

# Standardize numerics (fit on train only to avoid leakage)
scaler = None
if len(num_cols) > 0:
    scaler = StandardScaler()
    train[num_cols] = scaler.fit_transform(train[num_cols])
    test[num_cols] = scaler.transform(test[num_cols])

# Build categorical cardinalities list for embeddings
cat_cardinalities = [int(train[c].nunique()) for c in cat_cols]
cat_cardinalities


Categoricals (9): ['contact', 'default', 'education', 'housing', 'job', 'loan', 'marital', 'month', 'poutcome']
Numerics (7): ['age', 'balance', 'campaign', 'day', 'duration', 'pdays', 'previous']


[3, 2, 4, 2, 12, 2, 3, 12, 4]

In [13]:
# =========================
# DATASET
# =========================
class TabDataset(Dataset):
    def __init__(self, df, y=None, num_cols=None, cat_cols=None):
        self.num = df[num_cols].values.astype(np.float32) if num_cols else np.zeros((len(df),0), np.float32)
        self.cat = df[cat_cols].values.astype(np.int64) if cat_cols else np.zeros((len(df),0), np.int64)
        self.y = y.astype(np.float32) if y is not None else None

    def __len__(self):
        return len(self.num)

    def __getitem__(self, idx):
        if self.y is None:
            return self.num[idx], self.cat[idx]
        return self.num[idx], self.cat[idx], self.y[idx]


In [14]:
# =========================
# MODEL
# =========================
class TabularNN(nn.Module):
    def __init__(self, num_dim, cat_cardinalities, hidden_layers, dropout=0.2):
        super().__init__()
        self.has_cat = len(cat_cardinalities) > 0
        self.has_num = num_dim > 0

        # Embeddings
        if self.has_cat:
            emb_dims = []
            self.emb_layers = nn.ModuleList()
            for card in cat_cardinalities:
                emb_dim = int(min(50, max(4, round(1.6 * (card ** 0.56)))))  # heuristic
                self.emb_layers.append(nn.Embedding(card, emb_dim))
                emb_dims.append(emb_dim)
            emb_total = sum(emb_dims)
        else:
            emb_total = 0

        in_dim = (num_dim if self.has_num else 0) + emb_total

        layers = []
        prev = in_dim
        for h in hidden_layers:
            layers += [nn.Linear(prev, h), nn.ReLU(), nn.Dropout(dropout)]
            prev = h
        layers += [nn.Linear(prev, 1)]
        self.mlp = nn.Sequential(*layers)

    def forward(self, x_num, x_cat):
        feats = []
        if self.has_cat:
            embs = [emb(x_cat[:, i]) for i, emb in enumerate(self.emb_layers)]
            feats.append(torch.cat(embs, dim=1))
        if self.has_num:
            feats.append(x_num)
        x = torch.cat(feats, dim=1) if len(feats) > 1 else feats[0]
        logit = self.mlp(x).squeeze(1)
        return logit


In [15]:
# =========================
# TRAINING HELPERS
# =========================
def epoch_loop(model, loader, criterion, optimizer=None):
    is_train = optimizer is not None
    model.train() if is_train else model.eval()
    losses = []
    preds = []
    targs = []

    for batch in loader:
        if is_train:
            x_num, x_cat, y = batch
        else:
            try:
                x_num, x_cat, y = batch
            except:
                x_num, x_cat = batch
                y = None

        x_num = x_num.to(device)
        x_cat = x_cat.to(device)
        if y is not None:
            y = y.to(device)

        with torch.set_grad_enabled(is_train):
            logit = model(x_num, x_cat)
            prob = torch.sigmoid(logit)
            if y is not None:
                loss = criterion(logit, y)
            else:
                loss = None

        if is_train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if loss is not None:
            losses.append(loss.item())
            targs.append(y.detach().cpu().numpy())
        preds.append(prob.detach().cpu().numpy())

    preds = np.concatenate(preds) if len(preds)>0 else np.array([])
    y_true = np.concatenate(targs) if len(targs)>0 else None
    avg_loss = float(np.mean(losses)) if losses else None
    return avg_loss, preds, y_true

class EarlyStopper:
    def __init__(self, patience=10, mode="max", min_delta=1e-6):
        self.patience = patience
        self.mode = mode
        self.best = -np.inf if mode=="max" else np.inf
        self.count = 0
        self.min_delta = min_delta
        self.best_state = None

    def step(self, metric, model):
        improved = (metric > self.best + self.min_delta) if self.mode=="max" else (metric < self.best - self.min_delta)
        if improved:
            self.best = metric
            self.count = 0
            self.best_state = {k: v.cpu().clone() for k,v in model.state_dict().items()}
            return True
        else:
            self.count += 1
            return False

    def should_stop(self):
        return self.count >= self.patience


In [16]:
# =========================
# STRATIFIED K-FOLD TRAINING
# =========================
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_SEED)

oof = np.zeros(len(train), dtype=np.float32)
test_preds = np.zeros((len(test), N_SPLITS), dtype=np.float32)

metrics_rows = []

pos_weight = None
if USE_CLASS_WEIGHTS:
    pos_ratio = train[TARGET_COL].mean()
    # Avoid div by zero
    pos_weight = max(1e-6, (1.0 - pos_ratio) / max(1e-6, pos_ratio))
    pos_weight = torch.tensor([pos_weight], dtype=torch.float32, device=device)

for fold, (tr_idx, va_idx) in enumerate(skf.split(train[feature_cols], train[TARGET_COL])):
    print(f"\n===== Fold {fold} =====")
    tr_df = train.iloc[tr_idx].reset_index(drop=True)
    va_df = train.iloc[va_idx].reset_index(drop=True)

    tr_ds = TabDataset(tr_df, tr_df[TARGET_COL].values, num_cols, cat_cols)
    va_ds = TabDataset(va_df, va_df[TARGET_COL].values, num_cols, cat_cols)
    te_ds = TabDataset(test, None, num_cols, cat_cols)

    tr_loader = DataLoader(tr_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
    va_loader = DataLoader(va_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)
    te_loader = DataLoader(te_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

    model = TabularNN(num_dim=len(num_cols), cat_cardinalities=cat_cardinalities, hidden_layers=HIDDEN_LAYERS, dropout=DROPOUT).to(device)

    if pos_weight is not None:
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    else:
        criterion = nn.BCEWithLogitsLoss()

    optimizer = torch.optim.AdamW(model.parameters(), lr=BASE_LR, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.5, patience=3)

    early = EarlyStopper(patience=PATIENCE, mode="max")

    fold_log = []
    best_auc = -np.inf

    for epoch in range(1, EPOCHS+1):
        tr_loss, _, _ = epoch_loop(model, tr_loader, criterion, optimizer=optimizer)
        va_loss, va_pred, va_true = epoch_loop(model, va_loader, criterion, optimizer=None)
        va_auc = roc_auc_score(va_true, va_pred)

        scheduler.step(va_auc)

        fold_log.append(dict(epoch=epoch, train_loss=tr_loss, val_loss=va_loss, val_auc=float(va_auc), lr=float(optimizer.param_groups[0]['lr'])))

        if epoch % VERBOSE_EVERY == 0:
            print(f"Epoch {epoch:03d} | tr_loss={tr_loss:.5f} | va_loss={va_loss:.5f} | va_auc={va_auc:.6f} | lr={optimizer.param_groups[0]['lr']:.2e}")

        improved = early.step(va_auc, model)
        if improved:
            best_auc = va_auc

        if early.should_stop():
            print(f"Early stopping at epoch {epoch}. Best AUC: {best_auc:.6f}")
            break

    # load best weights
    model.load_state_dict(early.best_state)

    # Save fold artifacts
    fold_dir = FOLDS_DIR / f"fold_{fold}"
    fold_dir.mkdir(exist_ok=True)

    # Save metrics log
    log_df = pd.DataFrame(fold_log)
    log_df.to_csv(fold_dir / "train_log.csv", index=False)

    # Save model
    torch.save(model.state_dict(), fold_dir / "model.pth")

    # Validation predictions
    _, va_pred, va_true = epoch_loop(model, va_loader, criterion, optimizer=None)
    oof[va_idx] = va_pred.squeeze()

    # Test predictions
    _, te_pred, _ = epoch_loop(model, te_loader, criterion, optimizer=None)
    test_preds[:, fold] = te_pred.squeeze()

    # Make plots (ROC + learning curves)
    # ROC
    fig_roc, ax = plt.subplots()
    RocCurveDisplay.from_predictions(va_true, va_pred, ax=ax)
    ax.set_title(f"Fold {fold} ROC")
    fig_roc.savefig(fold_dir / "roc_curve.png", bbox_inches="tight")
    plt.close(fig_roc)

    # Learning curves
    fig_lc, ax = plt.subplots()
    ax.plot(log_df["epoch"], log_df["train_loss"], label="train_loss")
    ax.plot(log_df["epoch"], log_df["val_loss"], label="val_loss")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.set_title(f"Fold {fold} Loss")
    ax.legend()
    fig_lc.savefig(fold_dir / "loss_curve.png", bbox_inches="tight")
    plt.close(fig_lc)

    # AUC over epochs
    fig_auc, ax = plt.subplots()
    ax.plot(log_df["epoch"], log_df["val_auc"], label="val_auc")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("ROC AUC")
    ax.set_title(f"Fold {fold} Val AUC")
    ax.legend()
    fig_auc.savefig(fold_dir / "val_auc_curve.png", bbox_inches="tight")
    plt.close(fig_auc)

    metrics_rows.append(dict(fold=fold, best_val_auc=float(best_auc)))

# Aggregate metrics
metrics_df = pd.DataFrame(metrics_rows)
overall_auc = roc_auc_score(train[TARGET_COL].values, oof)
metrics_df.loc[len(metrics_df)] = dict(fold="OOF", best_val_auc=float(overall_auc))
metrics_df.to_csv(RUN_DIR / "metrics.csv", index=False)
print("\nOOF ROC AUC:", overall_auc)

# Save OOF
if SAVE_OOF:
    oof_df = pd.DataFrame({ID_COL: train[ID_COL].values, "oof": oof, TARGET_COL: train[TARGET_COL].values})
    oof_df.to_csv(RUN_DIR / "oof_predictions.csv", index=False)

# Ensemble test preds
test_pred_mean = test_preds.mean(axis=1)
sub = pd.DataFrame({ID_COL: test[ID_COL].values, TARGET_COL: test_pred_mean})
sub.to_csv(RUN_DIR / "submission.csv", index=False)
print("Submission saved to:", (RUN_DIR / "submission.csv").as_posix())

# Save per-fold and overall ROC plots combined
fig_all, ax = plt.subplots()
for fold in range(N_SPLITS):
    va_mask = np.zeros(len(train), dtype=bool)
    va_mask[list(np.where(metrics_df["fold"]==fold)[0])] = True  # placeholder (not used for curves)
# We instead compute ROC once on all OOF; also show histogram
RocCurveDisplay.from_predictions(train[TARGET_COL].values, oof, ax=ax)
ax.set_title("OOF ROC Curve")
fig_all.savefig(FIGS_DIR / "oof_roc_curve.png", bbox_inches="tight")
plt.close(fig_all)

# Histogram of OOF probs
fig_hist, ax = plt.subplots()
ax.hist(oof, bins=50)
ax.set_title("OOF Prediction Distribution")
ax.set_xlabel("Predicted probability")
fig_hist.savefig(FIGS_DIR / "oof_pred_hist.png", bbox_inches="tight")
plt.close(fig_hist)

# Save quick README for the run
with open(RUN_DIR / "README.txt", "w") as f:
    f.write(
        "This folder contains outputs for a single run.\n"
        "- metrics.csv: per-fold best AUC + OOF AUC\n"
        "- oof_predictions.csv: OOF probabilities with IDs and targets\n"
        "- submission.csv: ready for Kaggle submit\n"
        "- folds/*: per-fold model.pth, training logs, and plots (ROC, losses, AUC)\n"
        "- figs/*: overall figures (OOF ROC, hist)\n"
        "- models/*, logs/*, artifacts/*: reserved for extras\n"
    )

print("All artifacts saved under:", RUN_DIR.as_posix())


===== Fold 0 =====


/opt/homebrew/Caskroom/miniconda/base/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 001 | tr_loss=0.56081 | va_loss=0.48479 | va_auc=0.950139 | lr=1.00e-03


/opt/homebrew/Caskroom/miniconda/base/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 002 | tr_loss=0.48116 | va_loss=0.46048 | va_auc=0.953995 | lr=1.00e-03


...

Epoch 051 | tr_loss=0.37799 | va_loss=0.42771 | va_auc=0.962626 | lr=3.13e-05


/opt/homebrew/Caskroom/miniconda/base/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 052 | tr_loss=0.37824 | va_loss=0.42808 | va_auc=0.962653 | lr=1.56e-05
Early stopping at epoch 52. Best AUC: 0.962833


/opt/homebrew/Caskroom/miniconda/base/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)



OOF ROC AUC: 0.9625154905879971
Submission saved to: runs/2025-08-12_09-12-57/submission.csv
All artifacts saved under: runs/2025-08-12_09-12-57
